In [1]:
class PatientPhenotype:
    
    def __init__(self, eid, case, sex, yearBirth):
        
        self.eid = eid
        self.case = case
        self.sex = sex
        self.yearBirth = yearBirth
        
    def getEid(self):
        return self.eid
     
    def getCase(self):
        return self.case
    
    def getSex(self):
        return self.sex
    
    def getYearBirth(self):
        return self.yearBirth

        

In [2]:
patientsPhen = {}

f = open('C:\\Users\\Antonis\\Documents\\GitHub\\diplwmatikh\\data\\phenotype_euro.txt','r')
f.readline()
i = 0 
for line in f:
    patientsPhen[line.split()[0]] = PatientPhenotype(line.split()[0],line.split()[3],line.split()[1],line.split()[2])
        

In [4]:
yearsCase0 = {}
yearsCase1 = {}
yearsMenCase0 = {}
yearsMenCase1 = {}
yearsWomenCase0 = {}
yearsWomenCase1 = {}
for i in patientsPhen.keys():
    yearsCase0[patientsPhen[i].getYearBirth()] = 0
    yearsCase1[patientsPhen[i].getYearBirth()] = 0
    yearsMenCase0[patientsPhen[i].getYearBirth()] = 0
    yearsMenCase1[patientsPhen[i].getYearBirth()] = 0
    yearsWomenCase0[patientsPhen[i].getYearBirth()] = 0
    yearsWomenCase1[patientsPhen[i].getYearBirth()] = 0


for i in patientsPhen.keys():
    if int(patientsPhen[i].getCase()) == 0:
        yearsCase0[patientsPhen[i].getYearBirth()] = yearsCase0[patientsPhen[i].getYearBirth()] + 1
    else:
        yearsCase1[patientsPhen[i].getYearBirth()] = yearsCase1[patientsPhen[i].getYearBirth()] + 1
count = 0
for i in patientsPhen.keys():
    count+=1
    if int(patientsPhen[i].getSex()) == 1:
        if int(patientsPhen[i].getCase()) == 0:
             
                yearsMenCase0[patientsPhen[i].getYearBirth()] = yearsMenCase0[patientsPhen[i].getYearBirth()] + 1
        else:
                yearsMenCase1[patientsPhen[i].getYearBirth()] = yearsMenCase1[patientsPhen[i].getYearBirth()] + 1
               
    else:
        if int(patientsPhen[i].getCase()) == 0:
                yearsWomenCase0[patientsPhen[i].getYearBirth()] = yearsWomenCase0[patientsPhen[i].getYearBirth()] + 1
              
        else:
                yearsWomenCase1[patientsPhen[i].getYearBirth()] = yearsWomenCase1[patientsPhen[i].getYearBirth()] + 1
                

# men-women chart

In [99]:
axisX1 = []
axisY1 = []

for i in yearsCase1.keys():
    axisX1.append(i)
    
axisX1.sort()

for i in axisX1:
    axisY1.append(yearsCase1[i])
    
    
axisX0 = []
axisY0 = []

for i in yearsCase0.keys():
    axisX0.append(i)
    
axisX0.sort()

for i in axisX0:
    axisY0.append(yearsCase0[i])

In [100]:
import matplotlib.pyplot as plt

plt.plot(axisX0,axisY0,'r')
plt.plot(axisX1,axisY1,'g')
plt.show()

# men chart

In [101]:
axisXm1 = []
axisYm1 = []

for i in yearsMenCase1.keys():
    axisXm1.append(i)
    
axisXm1.sort()

for i in axisX1:
    axisYm1.append(yearsMenCase1[i])
    
    
axisXm0 = []
axisYm0 = []

for i in yearsMenCase0.keys():
    axisXm0.append(i)
    
axisXm0.sort()

for i in axisXm0:
    axisYm0.append(yearsMenCase0[i])

In [102]:
import matplotlib.pyplot as plt

plt.plot(axisXm0,axisYm0,'r')
plt.plot(axisXm1,axisYm1,'g')
plt.show()

# women chart


In [103]:
axisXw1 = []
axisYw1 = []

for i in yearsWomenCase1.keys():
    axisXw1.append(i)
    
axisXw1.sort()

for i in axisXw1:
    axisYw1.append(yearsWomenCase1[i])
    
    
axisXw0 = []
axisYw0 = []

for i in yearsWomenCase0.keys():
    axisXw0.append(i)
    
axisXw0.sort()

for i in axisX0:
    axisYw0.append(yearsWomenCase0[i])

In [104]:
import matplotlib.pyplot as plt

plt.plot(axisXw0,axisYw0,'r')
plt.plot(axisXw1,axisYw1,'g')
plt.show()

In [35]:
content = []
i = 0
import struct
import binascii
with open("E:\\chr11.bed", mode='rb') as f: # b is important -> binary
    for line in f:
        if i < 1:
            i += 1
         #   print(struct.unpack(("i" * (len(line))),line))
           # print(binascii.hexlify(line).decode('utf-8'))
            #print(line.strip().decode('utf-8'))
           # print(int.from_bytes(line),byteorder='big')
            print(line.strip())
        else:
            break
                

b"l\x1b\x01\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xf7\xff\xff\xff\xff\xff\xff\x7f\xfe\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x7f\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x7f\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xf7\xff\xdf\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x7f\xff\xff\xff\xff\xff\x7f\xff\xff\xff\xff\xbf\xff\xff\xff\xff\xff\xff\xff\xff\xdf\xff\xff\xdf\xff\x7f\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xfd\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\